## 用户生命周期模型：
主要可以分为三种类型：

### 1. 历史数据拟合模型
这个是最简单，就是根据往年的CLV打标签，然后用今年的标签匹配

### 2. BG/NBD概率模型
典型的Non-Contractual：n the non-contractual world, customers do go away, but they do so silently，这种假设用户的购买行为遵从一定的概率分布：
其中涉及到的几个重要指标：
- recency_i = 第一次消费距离上次消费的时间长度( 0 < recency_i < T_i)
- frequency_i = 客户重复购买商品的次数
    - freq = 0 => 购买过一次;freq = 1购买过两次
- T_i = 顾客i第一次消费到现在的时间长度

### 3. 机器学习模型
这个和正常的数据挖掘是一样的，通过跑模型来拟合，可以估计到个体层面
参考此路径的基础model：..\Project\Self Project\用户生命周期价值预测

In [ ]:
import pandas as pd
filePath = r'Online Retail Clean Data.xlsx'
df = pd.read_excel(filePath)

#计算三个feature
df['Buy_first'] = df.groupby(['CustomerID','StockCode'], as_index=False)['InvoiceDate'].transform('min')
df['Buy_last'] = df.groupby(['CustomerID','StockCode'], as_index=False)['InvoiceDate'].transform('max')
df['recency'] = (df['Buy_last'] - df['Buy_first']).dt.days

df['frequency'] = df.groupby(['CustomerID','StockCode'], as_index=False)['InvoiceNo'].transform('count')
df['frequency'] = df['frequency']-1

df['T'] = (df['InvoiceDate'].max()-df['Buy_first']).dt.days
df['monetary_value'] = df['Quantity'] * df['UnitPrice']

df['monetary_value'] = df.groupby(['CustomerID','StockCode'], as_index=False)['monetary_value'].transform('mean')
# df = df[['CustomerID', 'frequency', 'recency', 'T', 'monetary_value']]

In [ ]:
from lifetimes import BetaGeoFitter
bgf = BetaGeoFitter(penalizer_coef=1)
bgf.fit(df['frequency'], df['recency'], df['T'])
print(bgf)

# 可视化（频率/新进度）矩阵
from lifetimes.plotting import plot_frequency_recency_matrix
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(12,8))
plot_frequency_recency_matrix(bgf,T=1)

结论：我们可以看到，如果一个客户从你这里购买了80次，并且他们最近一次购买是在他们350天内的时候，那么他们就是你最好的客户(右下角)，也就是在接下来T期时间内，购买概率高，预测的交易数最多。
 你最冷的顾客是那些在右上角的:他们很快买了很多东西，我们已经好几个星期没见过他们了。

在(10,250)附近还有美丽的“尾巴”。这代表那些不经常购物的顾客，但我们最近见过他或她，所以他们可能会再次购买——我们不确定他们是已经死了还是只是在购物之间。

不过，一般来说，就是右下角本来意义不大，因为肯定这群购买频繁，时间久的是潜客；
值得召回的是中间“彗星尾巴”的人，这群人还处于“犹豫不定”状态。

In [ ]:
from lifetimes.plotting import plot_probability_alive_matrix
fig = plt.figure(figsize=(12,8))
plot_probability_alive_matrix(bgf)

t = 1
df['predicted_purchases'] = bgf.conditional_expected_number_of_purchases_up_to_time(t, 
                                  df['frequency'], df['recency'], df['T'])
df.sort_values(by='predicted_purchases', ascending=False).head()

In [ ]:
#评估模型效果方式一：模型验证重复购买频率
from lifetimes.plotting import plot_period_transactions
plot_period_transactions(bgf)

In [ ]:
#模型训练
# 现在将数据集划分为校准周期数据集和保持数据集。
# 这很重要，因为我们想要测试我们的模型如何对尚未看到的数据执行（就像机器学习实践中的交叉验证一样）

# 时间划分为三段: 最开始2010/12/1 -> 2010/12/8 -> 2011/2/23(结束)

# 训练集 - 2010/12/1 - 2010/12/8
# 验证集 - 2010/12/9 - 2011/2/23

from lifetimes.utils import calibration_and_holdout_data

summary_cal_holdout = calibration_and_holdout_data(df, 'CustomerID', 'InvoiceDate',
                                        calibration_period_end='2010-12-08',
                                        observation_period_end='2011-02-23' )   

summary_cal_holdout.head()

In [ ]:
#预测结果
from lifetimes.plotting import plot_calibration_purchases_vs_holdout_purchases
bgf.fit(summary_cal_holdout['frequency_cal'], summary_cal_holdout['recency_cal'], summary_cal_holdout['T_cal'])
plot_calibration_purchases_vs_holdout_purchases(bgf, summary_cal_holdout)

#结论：该图通过重复购买次数（x轴）对校准期内的所有客户进行分组，然后在保持期（y轴）中对其重复购买进行平均。
 橙线和蓝线分别表示模型预测和y轴的实际结果。正如我们所看到的，我们的模型能够非常准确地预测出样本中客户群的行为，
 模型低估了4次购买和5次购买后。

In [ ]:
#客户交易预测：
#根据客户历史记录，我们现在可以预测个人未来的购买情况：
t = 10
individual = df.loc[12347]
bgf.predict(t, individual['frequency'], individual['recency'], individual['T'])
#预测12347用户未来10天内购买商品为概率为0.00123

In [ ]:
#客户概率历史
#根据客户交易历史记录，我们可以根据我们训练的模型计算其存活的历史概率。
#例如，我们想看看我们最好的客户的交易历史，看看活着的可能性：
from lifetimes.plotting import plot_history_alive
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(12,8))
id = 14606
days_since_birth = 365
sp_trans = df.loc[df['CustomerID'] == id]
plot_history_alive(bgf, days_since_birth, sp_trans, 'InvoiceDate')

# 图二
fig = plt.figure(figsize=(12,8))
id = 15858
days_since_birth = 365
sp_trans = df.loc[df['CustomerID'] == id]
plot_history_alive(bgf, days_since_birth, sp_trans, 'InvoiceDate')

In [ ]:
# gamma-gamma模型估算客户终生价值
# 该模型有个重要前提：购买频次和购买金额无相关性
#模型相关性
returning_customers_summary = df[df['frequency']>0]
returning_customers_summary.shape[0]
returning_customers_summary[['frequency','monetary_value']].corr()

# 我们用来估计用户群的CLV的模型叫做Gamma-Gamma子模型，它依赖于一个重要的假设。
#  实际上，Gamma-Gamma子模型假设货币价值和购买频率之间没有关系。
#  在实践中，我们需要检查两个向量之间的Pearson相关性是否接近于0，才能使用这个模型。

In [ ]:
# gamma-gamma模型训练

from lifetimes import GammaGammaFitter
ggf = GammaGammaFitter(penalizer_coef = 0.1)

returning_customers_summary = returning_customers_summary[returning_customers_summary['monetary_value'] > 0]

ggf.fit(returning_customers_summary['frequency'],
        returning_customers_summary['monetary_value'])

# 这样我们可以估算每个客户的平均交易价值
ggf.conditional_expected_average_profit(
        returning_customers_summary['frequency'],
        returning_customers_summary['monetary_value']
    ).head()

In [ ]:
#使用DCF method计算CLV
# Discount Cash Flow, DCF 现金流折现，得到用户总体价值的当下估值。

bgf.fit(returning_customers_summary['frequency'], returning_customers_summary['recency'], returning_customers_summary['T'])

print(ggf.customer_lifetime_value(
    bgf, #the model to use to predict the number of future transactions
    returning_customers_summary['frequency'],
    returning_customers_summary['recency'],
    returning_customers_summary['T'],
    returning_customers_summary['monetary_value'],
    time=12, # months
    discount_rate=0.01 # monthly discount rate ~ 12.7% annually
).head(10))

In [ ]:
#BG-NBD模型用于客群分析
import numpy as np
from scipy.optimize import minimize

from scipy.special import gammaln

def negative_log_likelihood(params, x, t_x, T):
    if np.any(np.asarray(params) <= 0):
        return np.inf

    r, alpha, a, b = params

    ln_A_1 = gammaln(r + x) - gammaln(r) + r * np.log(alpha)
    ln_A_2 = (gammaln(a + b) + gammaln(b + x) - gammaln(b) -
           gammaln(a + b + x))
    ln_A_3 = -(r + x) * np.log(alpha + T)
    ln_A_4 = x.copy()
    ln_A_4[ln_A_4 > 0] = (
        np.log(a) -
        np.log(b + ln_A_4[ln_A_4 > 0] - 1) -
        (r + ln_A_4[ln_A_4 > 0]) * np.log(alpha + t_x)
    )
    
    delta =  np.where(x>0, 1, 0)
    
    log_likelihood = ln_A_1 + ln_A_2 + np.log(np.exp(ln_A_3) + delta * np.exp(ln_A_4))
    return -log_likelihood.sum()

scale = 1 / df['T'].max()
scaled_recency = df['recency'] * scale
scaled_T = df['T'] * scale


def _func_caller(params, func_args, function):
    return function(params, *func_args)

current_init_params = np.array([1.0, 1.0, 1.0, 1.0])
output = minimize(
    _func_caller,
    method="Nelder-Mead",
    tol=0.0001,
    x0=current_init_params,
    args=([df['frequency'], scaled_recency, scaled_T], negative_log_likelihood),
    options={'maxiter': 2000}
)

r = output.x[0]
alpha = output.x[1]
a = output.x[2]
b = output.x[3]

alpha /= scale

print("r = {}".format(r))
print("alpha = {}".format(alpha))
print("a = {}".format(a))
print("b = {}".format(b))

In [ ]:
#整体销量预测
from scipy.special import hyp2f1


# 现在通过以上代码预测未来52周单个用户的交易次数为0.542次，但计算E(x)为总的用户购买总次数，
#  这里不能简单的通过单个用户交易次数乘以总用户数得到，因为每个用户的第一次交易时间点不同。
def expected_sales_to_time_t(t):
    hyp2f1_a = r
    hyp2f1_b = b
    hyp2f1_c = a + b - 1
    hyp2f1_z = t / (alpha + t)
    hyp_term = hyp2f1(hyp2f1_a, hyp2f1_b, hyp2f1_c, hyp2f1_z)
    
    return ((a + b - 1) / (a - 1)) * (1-(((alpha / (alpha+t)) ** r) * hyp_term))


expected_sales_to_time_t(52)



In [ ]:
# Period of consideration is 39 weeks.
# T indicates the length of time since first purchase
n_s = (39*7 - df['T']).value_counts().sort_index()

n_s.head()

In [ ]:
#最后算出接下来78周用户的交易总次数为4156次。
forecast_range = np.arange(0, 78, 1/7.0)

def cumulative_repeat_transactions_to_t(t):
    expected_transactions_per_customer = (t - n_s.index).map(lambda x: expected_sales_to_time_t(x) if x > 0 else 0)
    expected_transactions_all_customers = (expected_transactions_per_customer * n_s).values
    return expected_transactions_all_customers.sum()

cum_rpt_sales = pd.Series(map(cumulative_repeat_transactions_to_t, forecast_range), index=forecast_range)

cum_rpt_sales.tail(10)

In [ ]:
#单个用户交易预测的条件
# 某用户过去38.86周内(T=38.86)交易两次(x=2)
#  第二次交易时间为30.43周(tx=30.43)的条件
#  计算得到该用户在未来39周将交易7.38次。
def calculate_conditional_expectation(t, x, t_x, T):
    first_term = (a + b + x - 1) / (a-1)
    hyp2f1_a = r + x
    hyp2f1_b = b + x
    hyp2f1_c = a + b + x - 1
    hyp2f1_z = t / (alpha + T + t)
    hyp_term = hyp2f1(hyp2f1_a, hyp2f1_b, hyp2f1_c, hyp2f1_z)
    second_term = (1 - ((alpha + T) / (alpha + T + t)) ** (r + x) * hyp_term)
    delta = 1 if x > 0 else 0
    denominator = 1 + delta * (a / (b + x - 1)) * ((alpha + T) / (alpha + t_x)) ** (r+x)
    return first_term * second_term / denominator


calculate_conditional_expectation(39, 2, 30.43, 38.86)

## 典型机器学习调参实例
以lightgbm为例